<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/newunitoperation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement a new unit operation in Python code
In this workbook we deonstrate how to implement a new unit operation in Python code.

In [1]:
!pip install neqsim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 13.1 MB/s eta 0:00:00


# Implementing a new unit operation in NeqSim
A unit operation is created by extending the unitop interface from the neqsim library.

In the following example we illustrate this be adding a simple compressor model (CompressorExample class). The method description of the compressor model is found in the following link:

http://www.jmcampbell.com/tip-of-the-month/2011/11/compressor-calculations-rigorous-using-equation-of-state-vs-shortcut-method/

In [52]:
from neqsim import jNeqSim
from neqsim.process.unitop import unitop
from jpype import JImplements, JOverride
import json
import math

class CompressorExample(unitop):
    def __init__(self):
        super().__init__()
        self.serialVersionUID = None
        self.name = ""
        self.inletstream = None
        self.outletstream = None
        self.power = None
        self.polytropic_efficiency = 77.0
        self.pressure_out = 100.0
        self.temperature_outlet = None
        self.gas_flow_rate = None
        self.temperature_inlet = None
        self.pressure_inlet = None

    def setInletStream(self, stream):
        self.inletstream = stream

    @JOverride
    def getOutletStream(self):
        return self.outletstream

    def setoutlet_pressure(self, presout):
        self.pressure_out = presout

    def setpolytropic_efficiency(self, polyeff):
        self.polytropic_efficiency = polyeff

    def run(self, uuid):
      self.serialVersionUID = uuid
      input_fluid = self.inletstream.getFluid().clone()
      self.pressure_inlet = self.inletstream.getPressure()
      self.gas_flow_rate = self.inletstream.getFlowRate('MSm3/day')
      self.temperature_inlet = self.inletstream.getTemperature()
      input_fluid.initProperties()
      kappa = input_fluid.getGamma2()
      MW = input_fluid.getMolarMass()
      z_inlet = input_fluid.getZ()
      n = 1.0/ (1.0 - (kappa-1.0)/kappa*1.0/(self.polytropic_efficiency/100.0))
      self.power = n/(n-1.0)*(input_fluid.getTemperature()+273.15)*z_inlet/(self.polytropic_efficiency/100.0)*(self.gas_flow_rate*1.0e6/24.0/3600.0)*(101325.0/288.15)*(math.pow(self.pressure_out/self.pressure_inlet, (n-1.0)/n)-1.0)
      self.temperature_outlet = (self.temperature_inlet)*math.pow(self.pressure_out/self.pressure_inlet, (n-1.0)/n)

    def toJson(self):
      data_dict = {
            "name": self.name,
            "power": self.power,
            "temperature_inlet": self.temperature_inlet,
            "temperature_outlet": self.temperature_outlet,
            "gas_flow_rate": self.gas_flow_rate,
            "pressure_inlet": self.pressure_inlet,
            "pressure_out": self.pressure_out,
            "polytropic_efficiency": self.polytropic_efficiency
      }
      return json.dumps(data_dict)


# Implement the compressor model in a NeqSim process simulation

In [54]:
from neqsim.thermo import fluid
from neqsim import jNeqSim

fluid1 = fluid("srk")  # create a fluid using the SRK-EoS
fluid1.addComponent("methane", 0.9)
fluid1.addComponent("CO2", 0.1)
fluid1.setMixingRule('classic')

stream1 = jNeqSim.processSimulation.processEquipment.stream.Stream(fluid1)
stream1.setFlowRate(10.0, "MSm3/day")
stream1.setPressure(50.0, 'bara')
stream1.setTemperature(20.0, 'C')
stream1.run()

compressor1 = CompressorExample()
compressor1.setInletStream(stream1)
compressor1.setName("Compressor 1")
compressor1.setpolytropic_efficiency(77.0)
compressor1.setoutlet_pressure(100.0)

stream2 = jNeqSim.processSimulation.processEquipment.stream.Stream('out stream', compressor1.getOutletStream())
stream2.setName('outlet stream')

example_process = jNeqSim.processSimulation.processSystem.ProcessSystem()
example_process.add(stream1)
example_process.add(compressor1)
example_process.add(stream2)

example_process.run()

TypeError: Ambiguous overloads found for constructor neqsim.processSimulation.processEquipment.stream.Stream(str,NoneType) between:
	public neqsim.processSimulation.processEquipment.stream.Stream(java.lang.String,neqsim.thermo.system.SystemInterface)
	public neqsim.processSimulation.processEquipment.stream.Stream(java.lang.String,neqsim.processSimulation.processEquipment.stream.StreamInterface)


In [48]:
json_report= str(jNeqSim.processSimulation.util.report.Report(example_process).generateJsonReport())
output = json.loads(json_report)
print(output['Compressor 1'])

{'name': 'Compressor 1', 'power': 20511052.95294938, 'temperature_inlet': 293.15, 'temperature_outlet': 349.4018612911362, 'gas_flow_rate': 10.000000000000002, 'pressure_inlet': 50.0, 'pressure_out': 100.0, 'polytropic_efficiency': 77.0}
